# Performing landscape level predictions

Mount drive, install package


In [ ]:
import sys
print("Python version")
print(sys.version)

Python version
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
from google.colab import drive
# drive.mount('/content/drive')
!pip install git+https://github.com/PatBall1/detectree2.git

  Cloning https://github.com/PatBall1/detectree2.git to /tmp/pip-req-build-vztr3c29
  Running command git clone --filter=blob:none --quiet https://github.com/PatBall1/detectree2.git /tmp/pip-req-build-vztr3c29
  Resolved https://github.com/PatBall1/detectree2.git to commit 2f29886bafcb63743a7f3ab596c5c3796ca4a243
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-install-3kcq93pe/detectron2_814ab9c3956248cf876dae13021793d0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-3kcq93pe/detectron2_814ab9c3956248cf876dae13021793d0
  Resolved https://github.com/facebookresearch/detectron2.git to commit 0df2d73d0013db7de629602c23cc120219b4f2b8
  Preparing metadata (setup.py) ... done


## Tiling
Tile up the entire orthomosaic to make predictions

## Create full site predictions

### Access pre-trained model

Download pre-trained model

There might be a cleaner way to do it with `pkg_resources` but I haven't figured this out yet

Combine predictions into full output

In [ ]:
import cv2
from PIL import Image
import os
import numpy as np
import urllib.request
import glob

# intake library and plugin
# import intake
# from intake_zenodo_fetcher import download_zenodo_files_for_entry

# geospatial libraries
import geopandas as gpd

from rasterio.transform import from_origin
import rasterio.features

import fiona

from shapely.geometry import shape, mapping, box
from shapely.geometry.multipolygon import MultiPolygon

# machine learning libraries
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer

# visualisation
import holoviews as hv
from IPython.display import display
# import geoviews.tile_sources as gts

# import hvplot.pandas
# import hvplot.xarray

# hv.extension('bokeh', width=100)

In [ ]:
%matplotlib inline

In [ ]:
filepath = '/content/DJI_20240504221329_0168_D.JPG'
im = cv2.imread(filepath)
display(Image.fromarray(im))

AttributeError: 'NoneType' object has no attribute '__array_interface__'

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.imshow(im)

In [ ]:
# Define the project main folder
data_folder = './forest-modelling-detectree'

# Set the folder structure
config = {
    'in_geotiff': os.path.join(data_folder, 'input','tiff'),
    'in_png': os.path.join(data_folder, 'input','png'),
    'model': os.path.join(data_folder, 'model'),
    'out_geotiff': os.path.join(data_folder, 'output','raster'),
    'out_shapefile': os.path.join(data_folder, 'output','vector'),
}

# List comprehension for the folder structure code
[os.makedirs(val) for key, val in config.items() if not os.path.exists(val)]

In [ ]:
# define the URL to retrieve the model
fn = 'model_final.pth'
url = f'https://zenodo.org/record/5515408/files/{fn}?download=1'

urllib.request.urlretrieve(url, config['model'] + '/' + fn)

In [ ]:
cfg = get_cfg()

# if you want to make predictions using a CPU, run the following line. If using GPU, hash it out.
cfg.MODEL.DEVICE='cuda'

# model and hyperparameter selection
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

### path to the saved pre-trained model weights
cfg.MODEL.WEIGHTS = config['model'] + '/model_final.pth'

# set confidence threshold at which we predict
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.15

#### Settings for predictions using detectron config

predictor = DefaultPredictor(cfg)

In [ ]:
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1], scale=1.5, instance_mode=ColorMode.IMAGE_BW)   # remove the colors of unsegmented pixels
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
image = cv2.cvtColor(v.get_image()[:, :, :], cv2.COLOR_BGR2RGB)
display(Image.fromarray(image))

In [ ]:

plt.imshow(image)

In [ ]:
mask_array = outputs['instances'].pred_masks.cpu().numpy()

# get confidence scores too
mask_array_scores = outputs['instances'].scores.cpu().numpy()

In [ ]:

num_instances = mask_array.shape[0]
mask_array_instance = []
output = np.zeros_like(mask_array)

mask_array_instance.append(mask_array)
output = np.where(mask_array_instance[0] == True, 255, output)
fresh_output = output.astype(np.float)
x_scaling = 140/fresh_output.shape[1]
y_scaling = 140/fresh_output.shape[2]
# this is an affine transform. This needs to be altered significantly.
transform = from_origin(int(filepath[-17:-11])-20, int(filepath[-10:-4])+120, y_scaling, x_scaling)

In [ ]:
mask_array = outputs['instances'].pred_masks.numpy()

# get confidence scores too
mask_array_scores = outputs['instances'].scores.numpy()

num_instances = mask_array.shape[0]
mask_array_instance = []
output = np.zeros_like(mask_array)

mask_array_instance.append(mask_array)
output = np.where(mask_array_instance[0] == True, 255, output)
fresh_output = output.astype(np.float)
x_scaling = 140/fresh_output.shape[1]
y_scaling = 140/fresh_output.shape[2]
# this is an affine transform. This needs to be altered significantly.
transform = from_origin(int(filepath[-17:-11])-20, int(filepath[-10:-4])+120, y_scaling, x_scaling)

output_raster = config['out_geotiff'] + '/' + 'predicted_rasters_' + filepath[-17:-4]+ '.tif'

new_dataset = rasterio.open(output_raster, 'w', driver='GTiff',
                                height = fresh_output.shape[1], width = fresh_output.shape[2], count = fresh_output.shape[0],
                                dtype=str(fresh_output.dtype),
                                crs='+proj=utm +zone=50 +datum=WGS84 +units=m +no_defs',
                                transform=transform)

new_dataset.write(fresh_output)
new_dataset.close()

In [ ]:
print(outputs)

In [ ]:
mask_array = outputs['instances'].pred_masks.cpu.numpy()

# get confidence scores too
mask_array_scores = outputs['instances'].scores.cpu.numpy()

num_instances = mask_array.shape[0]
mask_array_instance = []
output = np.zeros_like(mask_array)

mask_array_instance.append(mask_array)
output = np.where(mask_array_instance[0] == True, 255, output)
fresh_output = output.astype(np.float)

In [ ]:
# # Read input band with Rasterio

# with rasterio.open(output_raster) as src:
#     shp_schema = {'geometry': 'MultiPolygon','properties': {'pixelvalue': 'int', 'score': 'float'}}

#     crs = src.crs
#     for i in range(src.count):
#         src_band = src.read(i+1)
#         src_band = np.float32(src_band)
#         conf = mask_array_scores[i]
#         # Keep track of unique pixel values in the input band
#         unique_values = np.unique(src_band)
#         # Polygonize with Rasterio. `shapes()` returns an iterable
#         # of (geom, value) as tuples
#         shapes = list(rasterio.features.shapes(src_band, transform=src.transform))

#         if i == 0:
#             with fiona.open(config['out_shapefile'] + '/predicted_polygons_' + filepath[-17:-4] + '_' + str(0) + '.shp', 'w', 'ESRI Shapefile',
#                             shp_schema) as shp:
#                 polygons = [shape(geom) for geom, value in shapes if value == 255.0]
#                 multipolygon = MultiPolygon(polygons)
#                         # simplify not needed here
#                         #multipolygon = multipolygon_a.simplify(0.1, preserve_topology=False)
#                 shp.write({
#                           'geometry': mapping(multipolygon),
#                           'properties': {'pixelvalue': int(unique_values[1]), 'score': float(conf)}
#                            })
#         else:
#             with fiona.open(config['out_shapefile'] + '/predicted_polygons_' + filepath[-17:-4] + '_' + str(0)+'.shp', 'a', 'ESRI Shapefile',
#                             shp_schema) as shp:
#                 polygons = [shape(geom) for geom, value in shapes if value == 255.0]
#                 multipolygon = MultiPolygon(polygons)
#                         # simplify not needed here
#                         #multipolygon = multipolygon_a.simplify(0.1, preserve_topology=False)
#                 shp.write({
#                           'geometry': mapping(multipolygon),
#                           'properties': {'pixelvalue': int(unique_values[1]), 'score': float(conf)}
#                            })

In [ ]:
# load and plot polygons
in_shp = glob.glob(config['out_shapefile'] + '/*.shp')

poly_df = gpd.read_file(in_shp[0])

plot_vector = poly_df.hvplot(hover_cols=['score'], legend=False).opts(fill_color=None,line_color=None,alpha=0.5, width=800, height=600)

plot_vector

Remove invalid geometries before cleaning. Might be a way to fix them rather than remove all together but I think the ones with invalid geometries tend to be bad crowns anyway.

Clean overlapping crowns. For significant overlaps, the most confident crown is selected. IoU can be varied based on how strict you want the filtering to be.